In [1]:
# Same module but using numba for parallelization. Suitable for simulating large numbers of qubits (> 23)
# Numba somehow doesn't work when inside an imported module

%precision 4

import cmath
import numpy as np
from numpy import *
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

import time
from numba import jit, njit

import Circuit_ops
from Circuit_ops import zero_state, apply

from Circuit import Circuit
from Gates import Gates
from Error_dist import r
from Compiler import compile_gate

from numba import jit, njit

from Gate_bases import x, y, z, s_phi, identity

def zero_state(n):
    state = zeros(2 ** n, dtype=complex)
    state[0] = 1
    return state

# -- Helper functions for gate calculations -- 

# Phase addition and multiplication functions
def Multiply_np(a, b):
    return a * b

@njit(parallel=True)
def Multiply_njit(a, b):
    return a * b

def Add_np(a, b, angle):
    return cos(angle/2) * a - 1j * sin(angle/2) * b

@njit(parallel=True)
def Add_njit(a, b, angle):
    return cos(angle/2) * a - 1j * sin(angle/2) * b


# -- Apply gate to state --
def apply(gate, state, global_phase=False, N_thresh=22):
    
    # A shorthand for the original state
    a = state
    N = int(log2(len(state)))
    
    # A shorthand for the state flipped by Pauli operators
    b = copy(a)
    b = b.reshape([2] * N)
    
    if N > N_thresh:
        Add = Add_njit
        Multiply = Multiply_njit
        
    else: 
        Add = Add_np
        Multiply = Multiply_np
        
    for k in range(len(gate[0])):

        q = gate[0][k]
        basis = gate[1][k]
        angle = gate[2]
        
        if basis == identity:
            pass
        
        if basis == x:
            b = roll(b, 1, q)
        
        if basis == y:
            b = roll(b, 1, q)
            b = swapaxes(b, 0, q)
            b[0] = Multiply(b[0], -1j)
            b[1] = Multiply(b[1], 1j)
            b = swapaxes(b, 0, q)
            
        if basis == s_phi:
            phi = gate[3][k]
            b = roll(b, 1, q)
            b = swapaxes(b, 0, q)
            phase1 = cos(phi) + 1j * sin(phi)
            phase2 = cos(phi) - 1j * sin(phi)
            b[0] = Multiply(b[0], phase2)
            b[1] = Multiply(b[1], phase1)
            b = swapaxes(b, 0, q)
            
        if basis == z:
            b = swapaxes(b, 0, q)
            b[1] = Multiply(b[1], -1)
            b = swapaxes(b, 0, q)
    
    b = b.flatten()
        
    state = Add(a, b, angle)
    
    # Remove global phase (may be awkward if first amplitude is close to zero)
    if global_phase == False:
        state = state * exp(- 1j * cmath.phase(state[0]))
    return state

# ADD COLOUR FOR PHASE?
def prob_plot(state, title):
    N = int(log2(len(state)))
    
    for i in range(2 ** N):
        plt.plot(linspace(i,i,2), linspace(0, abs(state[i])**2,2), 'b', linewidth=5)
    if (title):
        plt.title(title)
    else: 
        plt.title('Probability plot')
    plt.show()
    return

def fidelity(final, ideal):
    
    F = sum(final * ideal)
    
    return abs(F)**2

def save_fidelities(fidelities, filename, N, n_gates, err, runs):
    with open(filename, 'w') as f:
        f.write(f'Results for {N} qubits, {n_gates} gates with max error = {err * 100}% over {runs} runs \n')
        for fidelity in fidelities:
            f.write("%s\n" % fidelity)
        f.close()
        
def read_fidelities(filename):
    with open(filename) as f:
        new_fids = []
        fidelities2 = f.read().splitlines()
        for i in range(len(fidelities2)-1):
            new_fids.append(float(fidelities2[i+1]))
    return new_fids


# -- The quantum circuit --
class Circuit:
    def __init__(self, name, N):
        # Circuit name
        self.name = name
        # Single-qubit over-rotation, phase-error, two-qubit-over-rotation
        self.errors = [.0, .0, .0]
        # Ideal gates used for plotting circuits
        self.ideal_gates = []
        # Noisy native gates
        self.noisy_gates = []
        # Initialize state to be zero state
        self.state = zero_state(N)
        # Number of qubits where numba is used. It seems numba doesn't work when we import it from a different module
        self.N_thresh = 99
    
    def set_state(self, state):
        self.state = state

    def set_errors(self, errors):
        self.errors = errors
        
    def compile_gates(self):
        errors = self.errors
        ideal_gates = self.ideal_gates

        noisy_gates = []

        for ideal_gate in ideal_gates:
            compiled_gate = compile_gate(ideal_gate, errors)

            # Synthesized gate
            if type(compiled_gate) == tuple:
                for noisy_gate in compiled_gate:
                    noisy_gates.append(noisy_gate)
                
            # Native gate
            else:
                noisy_gates.append(compiled_gate)
        
        self.noisy_gates = noisy_gates

    # Computes the final state given the initial state and circuit
    def compute(self):
        state = self.state
        self.compile_gates()

        for gate in self.noisy_gates:
            state = apply(gate, state, N_thresh=self.N_thresh)
        return state
    
    # Clear gates
    def clear_gates(self):
        self.ideal_gates = []
        self.noisy_gates = []
        
    # -- Ideal gates in a circuit --
    def S_phi(self, q, t, phi):
        self.ideal_gates.append(["S_phi", q, t, phi])
        return self

    def X(self, q, t):
        self.ideal_gates.append(["X", q, t, None])
        return self

    def Y(self, q, t):
        self.ideal_gates.append(["Y", q, t, None])
        return self
    
    def Z(self, q, t):
        self.ideal_gates.append(["Z", q, t, None])
        return self
    
    def XX(self, q1, q2, t):
        self.ideal_gates.append(["XX", [q1, q2], t, None])
        return self
    
    # -- Synthesized gates --
    def H(self, q):
        self.ideal_gates.append(["H", q, None, None])
        return self

    def CNOT(self, q1, q2):
        self.ideal_gates.append(["CNOT", [q1, q2], None, None])
        return self

    # -- Plot circuit for ideal gates --
    def plot_circuit(self):
        return

In [34]:
# 26 operations on 23 qubits
# Note: 1 "CNOT" = 5 native gate operations
# Time: 26s with njit

N = 20
k = 5       # Number of CNOTs --> (k+1)-qubit Bell state

circ = Circuit(f"5 CNOTs with {N} qubits", N).Y(0,pi/2)

index = 2 ** (N - 1)
for i in range(k):
    circ = circ.CNOT(i, i+1)
    index += 2 ** (N - 2 - i)

ideal_state = circ.compute()

# 20% errors
err = 0.1
circ.set_errors([err, err, err])

start = time.perf_counter()
final_state = circ.compute()

# Fidelity
print("Fidelity = ", abs((final_state[0] + final_state[index])/sqrt(2)) ** 2)
print("Time elapsed = ", time.perf_counter() - start, "s")

# Save memory
final_state = 0
ideal_state = 0

Fidelity =  0.8748756881737312
Time elapsed =  0.8798369149999417 s


In [7]:
# 25 operations on 25 qubits
# Note: 1 "CNOT" = 5 native gate operations
# Time: 122s with njit

N = 25

circ = Circuit(f"5 CNOTs with {N} qubits", N).Y(0,pi/2).CNOT(0, 6).CNOT(6, 7).CNOT(7, 8).CNOT(8, 9).CNOT(9, 10)

ideal_state = circ.compute()

# 20% errors
err = 0.2
circ.set_errors([err, err, err])

start = time.perf_counter()

final_state = circ.compute()

# Fidelity
print("Fidelity = ", fidelity(final_state, ideal_state))
print("Time elapsed = ", time.perf_counter() - start, "s")


# Save memory
final_state = 0
ideal_state = 0

Fidelity =  0.4872847396859132
Time elapsed =  57.16759820600009 s


In [3]:
# 5 operations (1 CNOT gate) on 27 qubits
# Note: 1 "CNOT" = 5 native gate operations
# Time taken: 310s with njit;

N = 27

n = 6
circ = Circuit(f"2 gates with {N} qubits", N).Y(0,pi/2).CNOT(0, n)

# 20% errors
err = 0.2
circ.set_errors([err, err, err])

start = time.perf_counter()

final_state = circ.compute()

# Fidelity
print("Fidelity = ", fidelity(final_state, ideal_state))
print("Time elapsed = ", time.perf_counter() - start, "s")

# Save memory
final_state = 0
ideal_state = 0

Fidelity =  0.9042774137323244
Time elapsed =  310.487747163 s


In [14]:
# 5 operations (1 CNOT gate) on 27 qubits
# Note: 1 "CNOT" = 5 native gate operations
# Time taken: 310s with njit;

N = 2

n = 1
circ = Circuit(f"2 gates with {N} qubits", N).Y(0,pi/2).CNOT(0, n)

# 20% errors
err = 0.02
circ.set_errors([err, err, err])

start = time.perf_counter()

final_state = circ.compute()

# Fidelity
print("Fidelity = ", final_state)
print("Time elapsed = ", time.perf_counter() - start, "s")

# Save memory
final_state = 0
ideal_state = 0

Fidelity =  [ 0.70366576+8.67361738e-19j -0.03794723-4.11471396e-03j
  0.03827126-4.92968658e-03j  0.7070035 +4.53281723e-02j]
Time elapsed =  0.0031555339999158605 s
